In [9]:
import pandas as pd
in_df = pd.read_csv("C:\\Users\\HP\\Desktop\\Data Analysis\\insurance.csv")
in_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [2]:
!pip install sqlalchemy psycopg2-binary pandas

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 4.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

class PostgreSQLManager:
    def __init__(self, username, password, host, port, database):
        self.conn_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'
        self.engine = create_engine(self.conn_string)
    
    def read_table(self, table_name):
        """Read entire table into a DataFrame"""
        try:
            df = pd.read_sql_table(table_name, self.engine)
            print(f"Table '{table_name}' read successfully.")
            return df
        except Exception as e:
            print(f"Error reading table: {e}")

    def write_dataframe(self, df, table_name, if_exists='replace'):
        """Write a DataFrame to the specified table"""
        try:
            df.to_sql(table_name, self.engine, if_exists=if_exists, index=False)
            print(f"DataFrame written to table '{table_name}' successfully.")
        except Exception as e:
            print(f"Error writing DataFrame: {e}")
    
    def inner_join(self, table1, table2, join_col):
        """Perform INNER JOIN on two tables and return result as DataFrame"""
        try:
            query = f"""
                SELECT *
                FROM {table1}
                INNER JOIN {table2}
                ON {table1}.{join_col} = {table2}.{join_col}
            """
            df = pd.read_sql_query(query, self.engine)
            print(f"INNER JOIN between '{table1}' and '{table2}' on '{join_col}' successful.")
            return df
        except Exception as e:
            print(f"Error performing join: {e}")
db = PostgreSQLManager(
    username='postgres',
    password='postgres',
    host='localhost',
    port='5432',
    database='postgres'
)
students_df = db.read_table('students')

Table 'students' read successfully.


In [7]:
students_df

,student_id,full_name,country,registered_on
0,101,Aditi Mehra,India,2023-06-12
1,102,Tom Richardson,USA,2022-11-04
2,103,Li Wei,China,2024-02-21
3,104,Sara Haddad,UAE,2023-08-09
4,106,Leo Müller,Germany,NaT
5,105,Victor Fernández,Argentina,2023-04-17


In [10]:
db.write_dataframe(in_df, 'insurance')

DataFrame written to table 'insurance' successfully.


In [11]:
joined_df = db.inner_join('students', 'course_enrollments', 'student_id')

INNER JOIN between 'students' and 'course_enrollments' on 'student_id' successful.


In [12]:
joined_df

,student_id,full_name,country,registered_on,enrollment_id,student_id,course_name,enrollment_date,score
0,101,Aditi Mehra,India,2023-06-12,201,101,Data Science 101,2023-06-15,85
1,102,Tom Richardson,USA,2022-11-04,202,102,Python for Web Dev,2022-11-05,78
2,103,Li Wei,China,2024-02-21,203,103,Machine Learning,2024-02-23,91
3,105,Victor Fernández,Argentina,2023-04-17,206,105,Python for Web Dev,2023-04-20,80
4,103,Li Wei,China,2024-02-21,207,103,AI Foundations,None,87
5,101,Aditi Mehra,India,2023-06-12,205,101,SQL for Analysts,2023-07-01,93
